# Prefetch example

In [2]:
import sys
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/system/tutorial/')
import lj_system
sys.path.insert(0, feasst.install_dir() + '/plugin/monte_carlo/tutorial/')
import analyze

def lj_system(box_length, cutoff):
    system = feasst.System()
    config = feasst.Configuration(feasst.args({
        "cubic_box_length": str(box_length),
        "particle_type": feasst.install_dir() + '/forcefield/data.lj',
        # "init_cells": "1.",
    }))
    config.set_model_param("cutoff", 0, cutoff);
    system.add(config)
    system.add(feasst.Potential(feasst.MakeLennardJones()))
    if system.configuration().domain().is_cell_enabled():
        #system.add_to_optimized(feasst.Potential(feasst.MakeLennardJones(), feasst.MakeVisitModelCell()))
        system.add_to_reference(feasst.Potential(feasst.MakeHardSphere(), feasst.MakeVisitModelCell()))
    return system

def mc(target_acceptance=0.2,
       tunable_param=0.185,
       density=0.85,
       cutoff=2,
       temperature=0.88,
       steps_per=int(1e6),
       ):
    box_length = 2*cutoff
    file_app = "_a" + str(target_acceptance) + "_r" + str(cutoff)
    num_particles=int(density*box_length**3)
    print('num_particles', num_particles)
    print('target_acceptance', target_acceptance)
    print('tunable_param', tunable_param)

    monte_carlo = feasst.Prefetch(feasst.args({"steps_per_check": "10000000"}))
    monte_carlo.activate_prefetch(False)
    monte_carlo.set(lj_system(box_length=box_length, cutoff=cutoff))
    monte_carlo.set(feasst.MakeMetropolis(feasst.args({
        "beta": str(1./temperature),
        "chemical_potential": "1.",
    })))
    monte_carlo.add(feasst.MakeTrialTranslate(feasst.args({
        "weight": "1.",
        "tunable_param": "2.",
        "tunable_param": str(tunable_param),
        "tunable_target_acceptance": str(target_acceptance),
        "tunable_percent_change": "0.01",
        # "num_steps": "4",
        # "reference_index": "0",
    })))
    monte_carlo.seek_num_particles(num_particles)
    analyze.add(monte_carlo,
        steps_per,
        proc=file_app,
        log="log"+file_app+".txt",
        )

    #equilibrate
    monte_carlo.attempt(int(1e6))

    monte_carlo.activate_prefetch(True)
    monte_carlo.add(feasst.MakeMeanSquaredDisplacement(feasst.args({
        "steps_per_update": "10000",
        "updates_per_origin": "1000",
        "file_name": "msd" + file_app + ".txt",
        "steps_per_write": str(int(1e5))
    })))
    monte_carlo.add(feasst.MakeCPUTime(feasst.args({
        "steps_per_update": str(steps_per),
        "steps_per_write": str(steps_per),
        "file_name": "cpu" + file_app + ".txt",
    })))
    monte_carlo.attempt(int(1e7))

mc()

num_particles 54
target_acceptance 0.2
tunable_param 0.185
